In [49]:
import numpy as np
import pandas as pd
from os import listdir
from glob import glob
import re
import copy

import pandas as pd
from tqdm import tqdm
from datetime import datetime
from operator import itemgetter
from time import mktime

from datetime import date

In [6]:
# Process HTML files to generate a csv of bikes at each station, and station locations

HTML_DIR = 'html_pages'
DATA_DIR = 'csv_files'

LAT_IDX = 0
LONG_IDX = 1

STAT_NAME = 0
STAT_ADDRESS = 1
STAT_BIKES = 2
STAT_DOCKS = 3

files = glob(HTML_DIR + '/*.html')
# num_files = len(files)
# print('Found {} files'.format(len(files)))

date_re = re.compile('.*stations_(\d{4}-\d{2}-\d{2}).*\.html')
time_re = re.compile('.*stations_\d{4}-\d{2}-\d{2}_(\d{2}:\d{2}:)\d{2}.*\.html')
station_re = re.compile('^var marker = new createMarker\(point, \"<div class=\'markerTitle\'><h3></h3></div>\
<div class=\'markerPublicText\'><h5>(\w.*)</h5></div>\
<div class=\'markerAddress\'>(\w.*)</div><div class=\'markerAvail\'>\
<div style=\'float: left; width: 50%\'><h3>(\d+)</h3>Bikes</div>\
<div style=\'float: left; width: 50%\'><h3>(\d+)</h3>Docks</div></div>\"')
latlong_re = re.compile('var point = new google\.maps\.LatLng\((.+), (.+)\);')

# Dictionary to store stations (fast lookup by lat/lon)
stations = dict()
station_id = 1
# List to store the bike values for each station in a series
bike_list = list()


file_counter = 0

for bike_idx, bike_filename in tqdm(enumerate(files),  total=len(files)):
    file_counter += 1

    date = str(date_re.match(bike_filename).groups(0)[0])
    time = str(time_re.match(bike_filename).groups(0)[0])
    time += '00'
    datetime_string = date + ' ' + time

    with open(bike_filename, 'r') as bike_file:
        for line in bike_file:
#             print line
#             Check for latitude and longitude
            match = latlong_re.match(line)
            if (match != None):
                latitude = float(latlong_re.match(line).groups()[LAT_IDX])
                longitude = float(latlong_re.match(line).groups()[LONG_IDX])
                latlon = (latitude, longitude)

            match = station_re.match(line)
            if (match != None):

                name = str(station_re.match(line).groups()[STAT_NAME])
                address = str(station_re.match(line).groups()[STAT_ADDRESS].replace('<br />', ', '))
#                 address.replace('<br />', ', ')
                bikes = int(station_re.match(line).groups()[STAT_BIKES])
                docks = int(station_re.match(line).groups()[STAT_DOCKS])

                if (latlon not in stations):
                    new_station = dict()
                    new_station['station_id'] = station_id
                    new_station['name'] = name
                    new_station['address'] = address
                    new_station['lat'] = latitude
                    new_station['lon'] = longitude
                    new_station['datetime'] = datetime_string
                    stations[latlon] = new_station
                    station_id += 1

                new_bike = dict()
                new_bike['station_id'] = stations[latlon]['station_id']
                new_bike['latlon'] = latlon
                new_bike['datetime'] = datetime_string
                new_bike['bikes'] = bikes
                new_bike['docks'] = docks
                bike_list.append(new_bike)
#                 print 'Found {} stations'.format(len(station_list))


print('Found {} stations'.format(len(stations)))
print('Found records from {} to {}'.format(bike_list[0]['datetime'], bike_list[-1]['datetime']))

# print('Converting station and bike data to csv file')

stations_df = pd.DataFrame.from_dict(stations, orient='index')
stations_df = stations_df[['station_id', 'name', 'address', 'lat', 'lon', 'datetime']]
stations_df.sort_values('station_id', ascending=True, inplace=True)
stations_df.to_csv(DATA_DIR + '/station_names.csv', index=False)

bikes_df = pd.DataFrame(bike_list)
bikes_df = bikes_df[['station_id', 'datetime', 'bikes', 'docks']]
bikes_df.to_csv(DATA_DIR + '/station_history_cumulative_unsorted.csv', index=False)

  1%|          | 427/42029 [00:09<16:09, 42.91it/s]/home/grayson/anaconda2/lib/python2.7/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 42029/42029 [14:39<00:00, 47.76it/s]


Found 76 stations
Found records from 2018-10-31 09:30:00 to 2018-09-28 18:00:00


In [43]:
fileIn = open('csv_files/station_history_cumulative_unsorted.csv', 'r')
unsortedEntries = fileIn.readlines()
fileIn.close()
stations = dict()

listToSort = list()
header = None
for line in unsortedEntries:
    try:
        split = line.split(',')
        # 2018-10-31 09:30:00
        date = datetime.strptime(split[1], "%Y-%m-%d %H:%M:%S")
        listToSort.append((int(split[0]), date, split[2], split[3]))
        stations[int(split[0])] = None
    except:
        print("Found {}".format(line))
        header = line

sortedEntries = sorted(listToSort, key=itemgetter(1,0))
fileOut = open('csv_files/station_history_cumulative.csv' ,'w')
fileOut.write(header)

for key in stations:
    stations[key] = open('csv_files/station_history_{}.csv'.format(key), 'w')
    stations[key].write(header)
    
for line in sortedEntries:
    writeLine = ','.join(map(str, line))
    stations[line[0]].write(writeLine)

    fileOut.write(writeLine)

for key in stations:
    stations[key].close()

fileOut.close()

Found station_id,datetime,bikes,docks



In [44]:
gapFile = open('csv_files/station_gaps.csv', 'w')

# find gaps
lastDate = sortedEntries[0][1]
for line in sortedEntries:
    currentDate = line[1]
    if mktime(currentDate.timetuple()) - mktime(lastDate.timetuple()) > 60:
        if  mktime(currentDate.timetuple()) - mktime(lastDate.timetuple()) > 60*60: # GAP
            gapFile.write(','.join(map(str, (lastDate, '', currentDate))) + '\n')
        elif mktime(currentDate.timetuple()) - mktime(lastDate.timetuple()) > 8*60: # GAP
            gapFile.write(','.join(map(str, (lastDate, currentDate))) + '\n')
        lastDate = currentDate

gapFile.close()

In [47]:
sortedStations = pd.read_csv('csv_files/station_history_cumulative.csv')
sortedStations.head()

,station_id,datetime,bikes,docks
0,1,2018-06-16 18:50:00,3,12
1,2,2018-06-16 18:50:00,9,6
2,3,2018-06-16 18:50:00,2,12
3,4,2018-06-16 18:50:00,14,5
4,6,2018-06-16 18:50:00,4,7


In [50]:
def extend(data):
    data['Minute'] = data.apply(getMinute, axis=1)
    data['Hour'] = data.apply(getHour, axis=1)
    data['Day'] = data.apply(getDay, axis=1)
    data['Month'] = data.apply(getMonth, axis=1)
    data['Year'] = data.apply(getYear, axis=1)
    data['Day of Week'] = data.apply(getDayOfWeek, axis=1)
    del data['datetime']
    data = data.reset_index(drop=True)


def getMinute(row):
    return int(row["datetime"].split(' ')[1].split(':')[1])

def getHour(row):
    return int(row["datetime"].split(' ')[1].split(':')[0])

def getDay(row):
    return int(row["datetime"].split('-')[2].split(' ')[0])

def getMonth(row):
    return int(row["datetime"].split('-')[1])

def getYear(row):
    return int(row["datetime"].split('-')[0])

#MONDAY=0, SUNDAY=6
def getDayOfWeek(row):
    return date(getYear(row), getMonth(row), getDay(row)).weekday()

cumulative = pd.read_csv('csv_files/station_history_cumulative.csv')
extend(cumulative)
cumulative.to_csv('csv_files/station_history_cumulative.csv')
for key in stations:
    station = pd.read_csv('csv_files/station_history_{}.csv'.format(key))
    extend(station)
    station.to_csv('csv_files/station_history_{}.csv'.format(key))

MIT License

Copyright (c) 2017 Tim Gasser

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.